In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog 
from sklearn.metrics import accuracy_score
import random
import sys
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.svm import SVC
import math
from sklearn.preprocessing import StandardScaler
import cvxpy as cp
import sys
from random import seed
np.random.seed(1)
seed(1)

In [3]:
def optimize_QMCM(xTrain,yTrain,C=10,C_weights='default',with_q=1,with_b=1,with_z=1):
    if C_weights == 'default':
      C_weights = np.ones((xTrain.shape[0],1))
    yTrain1 = np.where(yTrain==0,-1,yTrain)  
    #xTrain--->(N,D)
    #yTrain--->(N,1)
    N = xTrain.shape[0]
    D = xTrain.shape[1]
 
    if with_q==1:
        w = cp.Variable((D,1))
        b = cp.Variable((1,1))
        qi = cp.Variable((N,1))
        z = cp.Variable((D,1))
        if with_z==1:
          lterm = cp.sum(z) +C*(C_weights.T)@qi #h + C*cp.sum(Ciqi)
        else:
          lterm = C*(C_weights.T)@qi
        objective = cp.Minimize(lterm)   

        constraints = []
        if with_b==1:
            for i in range(N):
                constraints += [
                    yTrain1[i]*(np.matrix(xTrain[i])@w + b) + qi[i] >= 1,\
                    qi[i] >=0
                  ]
        else:
            for i in range(N):
                constraints += [
                    yTrain1[i]*(np.matrix(xTrain[i])@w) + qi[i] >= 1,\
                    qi[i] >=0
                  ]
        if with_z==1:
          for i in range(D):
              constraints += [
                  z[i] >= w[i],\
                  z[i] >= -w[i]
                ]
 
        prob = cp.Problem(objective,constraints)
        try:
          prob.solve()
        except:
          prob.solve(solver=cp.SCS,verbose=True)
        try:
          _w = w.value
          _b = b.value
          _qi = qi.value
          _z = z.value
        except:
          sys.exit('Not solved')
    else:
        w = cp.Variable((D,1))
        b = cp.Variable((1,1))
        qi = cp.Variable((N,1))
        z = cp.Variable((D,1))
        lterm = cp.sum(z)
        objective = cp.Minimize(lterm)   
 
        constraints = []
        if with_b==1:
            for i in range(N):
                constraints += [
                    yTrain1[i]*(np.matrix(xTrain[i])@w + b) >= 1
                  ]
        else:
            for i in range(N):
                constraints += [
                    yTrain1[i]*(np.matrix(xTrain[i])@w) >= 1
                  ]
 
        if with_z==1:
          for i in range(D):
              constraints += [
                  z[i] >= w[i],\
                  z[i] >= -w[i]
                ]
 
 
        prob = cp.Problem(objective,constraints)
        try:
          prob.solve()
        except:
          prob.solve(solver=cp.SCS,verbose=True)
        try:
          _w = w.value
          _b = b.value
          _qi = qi.value
          _z = z.value
        except:
          sys.exit('Not solved')
 
    yPred = np.zeros((N))
    wtx = np.zeros((N))
    for i in range(N):
        if with_b==1:
          _sum = (np.matrix(xTrain[i,:]))@_w + _b[0][0]
        else:
          _sum = (np.matrix(xTrain[i,:]))@_w
        wtx[i]=_sum
        if _sum>=0:
          yPred[i]=1
        else:
          yPred[i]=-1
 
    trainAcc=accuracy_score(yTrain1, yPred)
 
    if with_b==1:
      W = np.hstack((_b[0],_w.reshape(-1)))
    else:
      W = np.hstack((_b[0]*0,_w.reshape(-1)))
 
    try:
      qi_next = np.abs(_qi.reshape(-1))
    except:
      if _qi==None:
        qi_next = np.ones((N))
    return np.matrix(W),np.where(yPred==-1,0,yPred),trainAcc,qi_next

In [4]:
def classes(y,y_res):
    C1 = []
    C2 = []
    C3 = []
    C4 = []
    y = y.astype(int)
    y_res = y_res.astype(int)
    for i in range(len(y_res)): 
        if y[i] == 0 and y_res[i] == 0:
            C1.append(i)
        if y[i] == 1 and y_res[i] == 1:
            C2.append(i)
        if y[i] == 1 and y_res[i] == 0:
            C3.append(i)
        if y[i] == 0 and y_res[i] == 1:
            C4.append(i)
    return C1, C2, C3, C4

In [5]:
def quantize(inp,weig,scaler):
    inp_copy = inp
    if (inp.shape[1] < weig.shape[1]): 
        inp = np.hstack((inp,np.zeros((inp.shape[0],weig.shape[1]-inp.shape[1]))))
    inp_sc = scaler.transform(inp[:,1:])
    inp = np.hstack((np.ones((inp.shape[0],1)),inp_sc))
    for i in range(inp.shape[0]):
      for j in range(weig.shape[1]-inp_copy.shape[1]):
        inp[i,inp.shape[1]-j-1] = 0
    res = np.dot(inp,weig.T)
    res1 = np.dot(inp,weig.T)
    for j in range(res.shape[0]):
        if(res1[j][0] >= -0.00001):
            res[j][0] = 1
        if(res1[j][0] <= +0.00001):
            res[j][0] = 0
    return res

In [6]:
def choose_required_samples(XX,Xab,yab):
    return_index = list()
    for i in range(Xab.shape[0]):
        for j in range(XX.shape[0]):
            if np.array_equal(XX[j],Xab[i]):
                return_index.append(j)
    return return_index

In [7]:
class Neuron:
    def __init__(self,n):
        self.X = np.zeros((1,n)) #[bias weight, feature weights, weight from node A, weight from node B]
        self.inp = n #number of features
        self.A = None #node A
        self.B = None #node B
        self.Xab = np.zeros((1,n)) 
        self.yab = np.zeros((1))
        self.weight = 0 
        self.sc = 0 #scaler parameters stored in each node
  
    def insert(self,neuron_type, weight, Xab, yab): #insering child neurons    
        if neuron_type == 'A':
            self.A = Neuron(self.inp)
            self.A.Xab = Xab
            self.A.yab = yab
            self.A.weight = weight
            self.X = np.hstack((self.X, np.array([[weight]])))
        else:
            self.B = Neuron(self.inp)
            self.B.Xab = Xab
            self.B.yab = yab
            self.B.weight = weight
            self.X = np.hstack((self.X, np.array([[weight]])))
            
    def optimize_bottom_up(self,XX,bias_XX): #optimize the margin starting from the leaf nodes
        if self.A != None and self.B != None:
            # print("AB")
            y_1 = self.A.optimize_bottom_up(XX,bias_XX)
            y_2 = self.B.optimize_bottom_up(XX,bias_XX)
            inp_to_neuron = np.hstack((XX, y_1, y_2))
            training_inputs = inp_to_neuron[choose_required_samples(bias_XX,self.Xab,self.yab)]
            sc_ = StandardScaler().fit(training_inputs)
            training_inputs_sc = sc_.transform(training_inputs)
            self.sc = sc_
            W,yp,trAcc,qi = optimize_QMCM(training_inputs_sc,self.yab,C=100,with_q=0,with_b=1)
            self.X = W
            self.A.weight = W[0,-1]
            self.B.weight = W[0,-2]
            res = quantize(np.hstack((bias_XX, y_1, y_2)),self.X,self.sc)
            return res
        elif self.A != None:
            # print("A")
            y_1 = self.A.optimize_bottom_up(XX,bias_XX)
            inp_to_neuron = np.hstack((XX, y_1))
            training_inputs = inp_to_neuron[choose_required_samples(bias_XX,self.Xab,self.yab)]
            sc_ = StandardScaler().fit(training_inputs)
            training_inputs_sc = sc_.transform(training_inputs)
            self.sc = sc_
            W,yp,trAcc,qi = optimize_QMCM(training_inputs_sc,self.yab,C=100,with_q=0,with_b=1)
            self.X = W
            self.A.weight = W[0,-1]
            res = quantize(np.hstack((bias_XX, y_1)),self.X,self.sc)
            return res
        elif self.B != None:
            # print("B")
            y_2 = self.B.optimize_bottom_up(XX,bias_XX)
            inp_to_neuron = np.hstack((XX, y_2))
            training_inputs = inp_to_neuron[choose_required_samples(bias_XX,self.Xab,self.yab)]
            sc_ = StandardScaler().fit(training_inputs)
            training_inputs_sc = sc_.transform(training_inputs)
            self.sc = sc_
            W,yp,trAcc,qi = optimize_QMCM(training_inputs_sc,self.yab,C=100,with_q=0,with_b=1)
            self.X = W
            self.B.weight = W[0,-1]
            res = quantize(np.hstack((bias_XX, y_2)),self.X,self.sc)
            return res
        else:
            # print("no child node")
            inp_to_neuron = XX
            training_inputs = inp_to_neuron[choose_required_samples(bias_XX,self.Xab,self.yab)]
            sc_ = StandardScaler().fit(training_inputs)
            training_inputs_sc = sc_.transform(training_inputs)
            self.sc = sc_
            W,yp,trAcc,qi = optimize_QMCM(training_inputs_sc,self.yab,C=100,with_q=0,with_b=1)
            self.X = W
            res = quantize(bias_XX,self.X,self.sc)
            return res

    def calculate_till_depth_d(self,XX,bias_XX,d,count_depth): #for predicting with the neurons belonging to certain max depth
        count_depth = count_depth + 1
        if (count_depth == d): #stop calculation at this layer
            stop_cal = 1
        else:
            stop_cal = 0
        if (stop_cal == 0):
            if self.A != None and self.B != None:
                # print("AB")
                y_1 = self.A.calculate_till_depth_d(XX,bias_XX,d,count_depth)
                y_2 = self.B.calculate_till_depth_d(XX,bias_XX,d,count_depth)
#                 print("y_1:"+str(y_1))
#                 print("y_2:"+str(y_2))
                res = quantize(np.hstack((bias_XX, y_1, y_2)),self.X,self.sc)
                return res
            elif self.A != None:
                # print("A")
                y_1 = self.A.calculate_till_depth_d(XX,bias_XX,d,count_depth)
#                 print("y_1:"+str(y_1))
                res = quantize(np.hstack((bias_XX, y_1)),self.X,self.sc)
                return res
            elif self.B != None:
                # print("B")
                y_2 = self.B.calculate_till_depth_d(XX,bias_XX,d,count_depth)
#                 print("y_2:"+str(y_2))
                res = quantize(np.hstack((bias_XX, y_2)),self.X,self.sc)
                return res
            else:
                # print("no child node")
                res = quantize(bias_XX,self.X,self.sc)
                return res
        else:
            # print("we have stopped calculating")
            res = quantize(bias_XX,self.X,self.sc)
            return res

In [8]:
def accuracy(y_pred, y):
    count = 0
    if(len(y_pred) == len(y)):
        l = len(y)
        for i in range(l):
            if (y_pred[i] == y[i]):
                count+=1
        return count/l        
    else:
        print("Unequal lenghts of arrays!!")

In [9]:
def neuron_A(X,y,C1,C3,C4): # data set for neuron A
    n = X.shape[1]
    m = len(C1) + len(C3) + len(C4)
    y_A = np.zeros((m))
    X_A = np.zeros((m, n))
    for i in range(len(C1)):
        X_A[i] = X[C1[i]]
        y_A[i] = 0
    for i in range(len(C3)):
        X_A[i + len(C1)] = X[C3[i]]
        y_A[i + len(C1)] = 1
    for i in range(len(C4)):
        X_A[i + len(C1)+ len(C3)] = X[C4[i]]
        y_A[i + len(C1)+ len(C3)] = 0
    # print("X_A:")
    # print(X_A)
    # print("y_A:")
    # print(y_A)
    return X_A,y_A
def neuron_B(X,y,C2,C3,C4): # data set for neuron B
    n = X.shape[1]
    m = len(C2) + len(C3) + len(C4)
    y_B = np.zeros((m))
    X_B = np.zeros((m, n))
    for i in range(len(C2)):
        X_B[i] = X[C2[i]]
        y_B[i] = 0

    for i in range(len(C3)):
        X_B[i + len(C2)] = X[C3[i]]
        y_B[i + len(C2)] = 0

    for i in range(len(C4)):
        X_B[i + len(C2)+ len(C3)] = X[C4[i]]
        y_B[i + len(C2)+ len(C3)] = 1

    # print("X_B:")
    # print(X_B)
    # print("y_B:")
    # print(y_B)
    return X_B,y_B

In [10]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
def classifier(X,y,curr_Neuron,parent_Neuron):
    sc_ = StandardScaler().fit(X[:,1:])
    X_sc = sc_.transform(X[:,1:])
    X_sc = np.hstack((np.ones((X_sc.shape[0],1)),X_sc))
    W, yp, trAcc,qi = optimize_QMCM(X_sc[:,1:],y,C=100,C_weights='default',with_q=1)
    if curr_Neuron == parent_Neuron:
      curr_Neuron.Xab = X
      curr_Neuron.yab = y
    if (curr_Neuron == parent_Neuron) and (trAcc == 1):
      print("parent Neuron linearly separable")
    if trAcc == 1:
      print("Neuron linearly separable")
      curr_Neuron.Xab = X
      curr_Neuron.yab = y
    else:
      print("Neuron not linearly separable")
      W, yp, trAcc,qi = optimize_QMCM(X_sc[:,1:],y,C=C_fixed,with_q=1,with_b=1)
      qi = np.where(qi<0.1,1,qi)
      qi_previous=np.ones((X.shape[0]))
      cnt = 0
      while True and cnt<=100:
        C1,C2,C3,C4 = classes(y,yp)
        if len(C1)==0 or len(C2)==0:
          cnt = cnt+1 
          print('Got degenrate solution - cnt {}'.format(cnt))
          qi_previous = np.multiply(qi,qi_previous)
          W, yp, trAcc,qi = optimize_QMCM(X_sc[:,1:],y,C=C_fixed,C_weights=qi_previous,with_q=1,with_b=1)
          qi = np.where(qi<0.1,1,qi)
        else:
          print('Not degenerate solution')
          C1,C2,C3,C4 = classes(y,yp)
          break
      if cnt>100:
        sys.exit('Maximum count 100 reached..got only degenerate solutions in 100 reps')       
      print('\nC1 {} \nC2 {} \nC3 {} \nC4 {}'.format(C1,C2,C3,C4))
      if len(C3)!= 0:
          print('A')
          X_A, y_A = neuron_A(X,y,C1,C3,C4)
          curr_Neuron.insert('A', 0, X_A, y_A)
          classifier(X_A,y_A,curr_Neuron.A,parent_Neuron)
      if len(C4)!= 0:
          print('B')
          X_B, y_B = neuron_B(X,y,C2,C3,C4)
          curr_Neuron.insert('B', 0, X_B, y_B)
          classifier(X_B,y_B,curr_Neuron.B,parent_Neuron)

In [12]:
##@title
import os
from scipy.io import loadmat
from sklearn.model_selection import GridSearchCV
from collections import Counter

def transform_data(df):
  df["y"] = df["class"].apply(lambda x: 1 if (x=="present") else 0)
  df.drop(columns = ["class"], inplace=True)
  return df

dftrain = pd.read_csv('data/heart-statlog_csv.csv', header = 0)

dftrain = transform_data(dftrain)
print(f"data size = {dftrain.shape}")
Xtrain = dftrain.iloc[:,:-1].to_numpy()

ytrain = dftrain.iloc[:,-1].to_numpy()
n = Xtrain.shape[0]
i = 0
k = 1
r = int(n/10)
sum = 0

li = list()
li.append(str(Xtrain.shape).replace(',','/')) 
li.append(str(Counter(ytrain.tolist())).replace(',','/'))
names = ['#no', 'data', 'class']
svm_test_acc_fold = list()
svm_train_acc_fold = list()
for C_fixed in [0.01,0.1,1,10]:
  try:
    test_acc_fold = list()
    train_acc_fold = list()
    
    while(i < k):
      Xtra1 = Xtrain[0 : r*i]
      Ytra1 = ytrain[0 : r*i]
      Xtra2 = Xtrain[r*(i+1) : ]
      Ytra2 = ytrain[r*(i+1) : ]
      Xtra = np.concatenate((Xtra1, Xtra2), axis=0)
      Ytra = np.concatenate((Ytra1, Ytra2), axis=0)
      Xtest = Xtrain[r*i : r*(i+1)]
      Ytest = ytrain[r*i : r*(i+1)]
      fold_no=i
      
      X_train1,y_train1,X_test1,y_test1 = Xtra,Ytra,Xtest,Ytest
      
      curr_Neuron = Neuron(X_train1.shape[1])
      parent_Neuron = curr_Neuron
      classifier(X_train1, y_train1, curr_Neuron, parent_Neuron)
      results = parent_Neuron.optimize_bottom_up(X_train1[:,1:],X_train1)
      acc = []
      depth = 1
      depth_max = 5
      while(depth < depth_max):
          a = parent_Neuron.calculate_till_depth_d(X_train1[:,1:],X_train1,depth,0)
          acc = np.concatenate((acc, [accuracy(a, y_train1)]))
          if depth == depth_max-1:
              if accuracy(a, y_train1) < 1:
                  depth_max = depth_max + 1
          depth = depth + 1

      test_acc = []
      test_depth = 1
      while(test_depth < depth_max):
          a = parent_Neuron.calculate_till_depth_d(X_test1[:,1:],X_test1,test_depth,0)
          test_acc = np.concatenate((test_acc, [accuracy(a, y_test1)]))
          test_depth = test_depth + 1
      print("###train accuracy###")
      print(acc)
      print("###########")
      print("@@@test accuracy@@@")
      print(test_acc)
      print("@@@@@@@@@@")

      #plotting train and test accuracy vs the depth of the tree
      plt.figure(figsize=(10,5))
      m = acc.shape[0]
      xx = np.array(list(range(0,m)))
      xx = xx.reshape(m,1)
      plt.subplot(1, 2, 1)
      plt.plot(xx, acc.reshape(m,1), label = "Accuracy")
      plt.title('training data')
      plt.legend()

      mm = test_acc.shape[0]
      plt.subplot(1, 2, 2)
      plt.plot(xx, test_acc.reshape(mm,1), label = "Accuracy")
      plt.title('test data')
      plt.legend()

      plt.savefig('/content/{}_dataset_{}/C_{}_fold_{}.png'.format(dir_name,dataset,C_fixed,i))
      plt.show()

      train_acc_fold.append(acc[np.where(test_acc==max(test_acc))[0][0]])
      test_acc_fold.append(max(test_acc))

    li.append(str(np.mean(np.array(train_acc_fold)))+str('±')+str(np.std(np.array(train_acc_fold))))
    li.append(str(np.mean(np.array(test_acc_fold)))+str('±')+str(np.std(np.array(test_acc_fold))))

    names.extend(['train acc mean±std C = {}'.format(C_fixed)])
    names.extend(['test acc mean±std C = {}'.format(C_fixed)])

  except:
    pass
print('SVM..')
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
            'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
            'kernel': ['rbf']} 
while(i < k):
  Xtra1 = Xtrain[0 : r*i]
  Ytra1 = ytrain[0 : r*i]
  Xtra2 = Xtrain[r*(i+1) : ]
  Ytra2 = ytrain[r*(i+1) : ]
  Xtra = np.concatenate((Xtra1, Xtra2), axis=0)
  Ytra = np.concatenate((Ytra1, Ytra2), axis=0)
  Xtest = Xtrain[r*i : r*(i+1)]
  Ytest = ytrain[r*i : r*(i+1)]
  fold_no=i
  
  X_train1,y_train1,X_test1,y_test1 = Xtra,Ytra,Xtest,Ytest
  X_Tr,y_Tr,X_Te,y_Te = Xtra,Ytra,Xtest,Ytest
  sc_ = StandardScaler().fit(X_Tr[:,1:])
  X_sc = sc_.transform(X_Tr[:,1:])
  X_Tr = X_sc
  X_sc = sc_.transform(X_Te[:,1:])
  X_Te = X_sc

  grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 0)
  grid.fit(X_Tr, y_Tr)
  grid_predictions_Te = grid.predict(X_Te)
  grid_predictions_Tr = grid.predict(X_Tr)

  svm_train_acc_fold.append(accuracy_score(grid_predictions_Tr,y_Tr))
  svm_test_acc_fold.append(accuracy_score(grid_predictions_Te,y_Te))

li.append(str(np.mean(np.array(svm_train_acc_fold)))+str('±')+str(np.std(np.array(svm_train_acc_fold))))
li.append(str(np.mean(np.array(svm_test_acc_fold)))+str('±')+str(np.std(np.array(svm_test_acc_fold))))
names.extend(['SVM train acc mean±std'])
names.extend(['SVM test acc mean±std'])
np.savetxt("/content/{}_dataset_{}/tree_vs_svm.csv".format(dir_name,dataset),np.array(li).reshape(1,len(li)),fmt= '%s',delimiter=',',\
        header=','.join(names))


data size = (270, 14)
Neuron not linearly separable
Not degenerate solution

C1 [0, 2, 5, 11, 12, 14, 15, 18, 24, 25, 26, 27, 28, 30, 35, 36, 39, 41, 44, 45, 46, 47, 49, 50, 51, 52, 56, 58, 59, 61, 63, 69, 72, 73, 75, 79, 82, 84, 86, 87, 88, 91, 96, 97, 98, 100, 101, 105, 108, 109, 111, 114, 116, 122, 123, 124, 125, 126, 127, 128, 130, 135, 138, 139, 140, 141, 143, 146, 147, 152, 156, 157, 158, 161, 163, 167, 168, 169, 170, 171, 179, 182, 184, 185, 187, 188, 189, 192, 195, 197, 201, 205, 209, 211, 212, 214, 215, 217, 220, 224, 226, 227, 228, 229, 232, 233, 236, 240] 
C2 [1, 3, 6, 7, 8, 9, 13, 17, 21, 22, 23, 32, 34, 38, 43, 48, 53, 54, 55, 62, 65, 67, 70, 76, 77, 78, 80, 81, 89, 90, 92, 93, 95, 99, 102, 113, 115, 117, 118, 120, 121, 129, 132, 133, 136, 144, 149, 151, 154, 155, 159, 162, 164, 166, 174, 175, 176, 177, 181, 183, 186, 193, 194, 196, 199, 200, 203, 204, 206, 208, 210, 213, 218, 219, 222, 223, 230, 234, 237] 
C3 [10, 19, 20, 29, 31, 40, 42, 64, 66, 68, 74, 83, 85, 94, 103, 1

<ipython-input-3-042c8d583e11>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if C_weights == 'default':


Got degenrate solution - cnt 2
Not degenerate solution

C1 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163] 
C2 [109, 113, 121, 122, 129, 130, 132, 133, 138] 
C3 [108, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 123, 124, 125, 126, 127, 128, 131, 134, 135, 136, 137] 
C4 [10, 21, 27, 57, 71, 75, 79, 102, 151, 160]
A
Neuron not linearly separable
Got degenrate solution - cnt 1


<ipython-input-3-042c8d583e11>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if C_weights == 'default':


Got degenrate solution - cnt 2
Got degenrate solution - cnt 3
Got degenrate solution - cnt 4
Not degenerate solution

C1 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18, 19, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 51, 52, 53, 54, 55, 56, 57, 59, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 109, 110, 111, 112, 113, 114, 117, 118, 119, 120, 121, 122, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154] 
C2 [125, 126, 129, 130, 132, 134, 135, 136, 138] 
C3 [123, 124, 127, 128, 131, 133, 137, 139, 140, 141, 142, 143, 144] 
C4 [6, 15, 17, 20, 27, 32, 45, 49, 50, 58, 61, 95, 104, 108, 115, 116]
A
Neuron not linearly separable
Got degenrate solution - cnt 1


...

Neuron linearly separable
B
Neuron linearly separable
B
Neuron linearly separable
B
Neuron not linearly separable
Not degenerate solution

C1 [3, 5, 6, 10, 11, 12, 13, 14] 
C2 [15, 16, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41] 
C3 [17, 18, 26] 
C4 [0, 1, 2, 4, 7, 8, 9]
A
Neuron linearly separable
B
Neuron not linearly separable
Not degenerate solution

C1 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26] 
C2 [29] 
C3 [27, 28, 30, 31, 32, 33] 
C4 []
A
Neuron not linearly separable
Got degenrate solution - cnt 1


<ipython-input-3-042c8d583e11>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if C_weights == 'default':


Not degenerate solution

C1 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26] 
C2 [28] 
C3 [27, 29, 30, 31, 32] 
C4 [20, 24]
A
Neuron not linearly separable
Got degenrate solution - cnt 1
Got degenrate solution - cnt 2


<ipython-input-3-042c8d583e11>:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if C_weights == 'default':


Not degenerate solution

C1 [0, 2, 4, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 30] 
C2 [25, 26, 27, 28, 29] 
C3 [] 
C4 [1, 3, 5, 6, 9, 10, 23, 24, 31]
B
Neuron linearly separable
B
Neuron linearly separable
###train accuracy###
[0.87654321 0.79423868 0.83950617 0.9218107  0.95473251 0.97942387
 1.        ]
###########
@@@test accuracy@@@
[0.77777778 0.74074074 0.62962963 0.7037037  0.7037037  0.62962963
 0.62962963]
@@@@@@@@@@
SVM..
